In [1]:
import misc

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from logparser import LogReader, PosReader

# Prepare the data

In [ ]:
# Load log files
logs = []

logs.append(misc.getLogDictionnary(
    "Google Pixel 7",
    "../.data/static/gnss_log_GooglePixel7_2023_02_17_09_55_01.txt"
))

logs.append(misc.getLogDictionnary(
    "OnePlus Nord 2",
    "../.data/static/gnss_log_OnePlusNord2_2023_03_03_10_55_04.txt"
))

logs.append(misc.getLogDictionnary(
    "Samsung A52",
    "../.data/static/gnss_log_SamsungA52_2023_03_17_14_12_30.txt"
))

logs.append(misc.getLogDictionnary(
    "Xiaomi 11T",
    "../.data/static/gnss_log_Xiaomi11T_2023_03_03_09_56_06.txt"
))

# Parse log - This might take a while...
for log in logs:
    log['content'] = LogReader(log['filepath'], ['Raw', 'Fix'])

In [ ]:
# Set reference position
filepath = "../example_data/NMND17420010S_2023-08-01_08-24-35.pos"
ref = PosReader(filepath)

In [ ]:
# Compute ENU errors
ref_enu = [61.449219, 23.856267, 166.0]
for log in logs:
    log['content'].fix[["east", "north", "up"]] = log['content'].fix.apply(
        lambda row: misc.convert2ENU(row['latitude'], row['longitude'], row['altitude'], ref_enu), 
        axis='columns', result_type='expand')
    
    log['content'].fix[["2D_error"]] = log['content'].fix.apply(
        lambda row: misc.getHorizontalError(row['east'], row['north']), 
        axis='columns', result_type='expand')
    
    log['content'].fix[["3D_error"]] = log['content'].fix.apply(
        lambda row: misc.get3DError(row['east'], row['north'], row['up']), 
        axis='columns', result_type='expand')

# Results

In [ ]:
center = [61.449219, 23.856267]
#extent = [center[1] - 0.02, center[1] + 0.005, center[0] - 0.005, center[0] + 0.005]
extent = [center[1] - 0.001, center[1] + 0.006, center[0] - 0.001, center[0] + 0.002]

plotMap(locations=locations, extent=extent, scale=17)